In [1]:
# !pip install transformers
# !pip install torchmetrics

In [2]:
import pandas as pd 
import torch
import numpy as np
from transformers import BertTokenizer
from torch import nn
from transformers import BertModel
from tqdm import tqdm
from torchmetrics import AUROC
import torch.nn.functional as F

In [3]:
from lstm_utils import *

In [4]:
input_path = '../data/'
df = pd.read_csv(input_path+'user_top_3_data.csv')
df.fillna(' ', inplace=True)
df.head(2)

,user_no,key_label,input_1,input_2,input_3
0,2321,0,News highlights Top global markets news of th...,News highlights Top global markets news of th...,Intel to buy optimization software business Gr...
1,1168,0,Shares Of Penny Stock Tonix Pharma Tick Higher...,Tonix Pharmaceuticals to Participate in the No...,Tonix Pharmaceuticals to Participate in the No...


In [5]:
length = int(df['input_1'].apply(len).mean())
length = np.min([512, length])
print('length', length)

length 439


In [6]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

In [7]:
print(len(df_train),len(df_val), len(df_test))
df_val = df_val.iloc[:440,:]
df_test = df_test.iloc[:440, :]
print(len(df_train),len(df_val), len(df_test))

3580 447 448
3580 440 440


In [8]:
df_train.shape

(3580, 5)

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case = True)
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
model = BertClassifier_LSTM(bert_model, bert_freeze=False)

In [11]:
auroc = AUROC(num_classes=2)

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [12]:
train_dataset, val_dataset, test_dataset = Dataset_lstm(df_train, length, tokenizer), Dataset_lstm(df_val, length, tokenizer), Dataset_lstm(df_test, length, tokenizer)

In [13]:
EPOCHS = 10
LR = 1e-5

In [14]:
train_model(model, train_dataset, val_dataset, auroc, LR, EPOCHS)

len(train_data_loader) 358
device ~~~~~~~~~~  cuda


  8%|▊         | 30/358 [00:08<01:26,  3.80it/s]/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
100%|██████████| 358/358 [01:35<00:00,  3.76it/s]


Epochs: 1 | Train Loss:  0.068                 | Train Accuracy:  0.594                 | Train AUC:  0.518                | Val Loss:  0.068                 | Val Accuracy:  0.570                | Val AUC:  0.545


In [15]:
auroc = AUROC(num_classes=2)

evaluate_model(model, test_dataset, auroc)

device ~~~~~~~  cuda
Test Accuracy:  0.602             | Test AUC:  0.491
